In [ ]:
import pandas as pd
import tensorflow as tf
import regex as re
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
def preprocess_dataframe(df):

  df.fillna(value=0, inplace=True) #Reemplazamos NAN por 0, ya que NAN rompe a Tensorflow

  #Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
  #Estos particularmente son whitespace, coma y parentesis por ejemplo.
  df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                    'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                    'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

  df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
  df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won.

  df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                  format='%m/%d/%Y')
  df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df = df[df['Opportunity_ID'] != 9773] #Hardcodeo este filtrado porque el id 9773 tiene mal cargada la fecha de delivery end, dando una diferencia de 200 anios xd"

  #Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
  #estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
  #una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
  #la chance de completar la operacion.
  df['Delta_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
  df.loc[:, 'Delta_Time'] = df['Delta_Time'].dt.days
  df['Delta_Time'] = df['Delta_Time'].replace({np.nan:10.0}) #Reemplazo con 10 porque los que no tienen fecha final ganan el 60%, y el analisis de los datos da que el 60% es maso a los 10 dias. Asi no jodo el resto de los datos
  df['Delta_Time'] = df.groupby('Opportunity_ID')['Delta_Time'].transform('max')

  #Pasamos todo a dolares
  currency_conversion = {'AUD':0.707612, 'EUR':1.131064, 'GBP':1.318055, 'JPY':0.008987, 'USD':1.0}
  df['Total_Taxable_Amount_Currency'] = df[['Total_Taxable_Amount_Currency']].replace(currency_conversion)
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount_Currency'] * df['Total_Taxable_Amount']

  #Modifico la columna Brand para que en vez de decir que marca es, solo diga
  #si tiene o no marca. Es importante aclarar que verificamos que siempre que una oportunidad
  #tiene un producto con marca entonces todos sus productos tienen marca. Esto se cumple
  #tanto en el set de entrenamiento como en el de test, por lo tanto al hacer drop_duplicates
  #no nos va a pasar nunca el caso donde nos pudieramos quedar con una entrada de producto
  #sin marca mientras que algun otro producto si tuviera, ya que confirmamos que o todos tienen
  #marca o ninguno tiene.
  df.loc[df['Brand'] == 'None', 'Brand'] = 'No'
  df.loc[df['Brand'] != 'No', 'Brand'] = 'Yes'

  #Agrego una columna que indique la cantidad de productos que tiene esa
  #oportunidad
  df['Product'] = 1
  df['Product_Amount'] = df.groupby('Opportunity_ID')['Product'].transform(lambda x: x.sum())

  #Agrego una columna que indica si el owner de la cuenta es el mismo que el de la oportunidad
  #o no
  df['Same_Owner'] = (df['Account_Owner'] == df['Opportunity_Owner'])
  df['Same_Owner'] = df['Same_Owner'].replace({False:'No', True:'Yes'})

  #Agrego una columna que indica si tiene o no fecha de expiracion
  df['Quote_Expiry_Date'] = (df['Quote_Expiry_Date'] != 'NaT')
  df.rename(columns={'Quote_Expiry_Date':'Has_Expiry_Date'}, inplace=True)
  df['Has_Expiry_Date'] = df['Has_Expiry_Date'].replace({True:'Yes',False:'No'})

  #Reemplazo las 4 columnas de aprobacion por solo 2 columnas que indiquen si tuvo la aprobacion
  #de delivery y burocratica o no. Recalco que si nunca la necesito seria equivalente a si
  #la necesito y la consiguio.
  df['Delivery_Approved'] = df['Pricing_Delivery_Terms_Quote_Appr'] + df['Pricing_Delivery_Terms_Approved']
  df['Delivery_Approved'] = df['Delivery_Approved'].replace({0:1, 1:0, 2:1})
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_0_Approval'] + df['Bureaucratic_Code_0_Approved']
  df['Bureaucratic_Code_Approved'] = df['Bureaucratic_Code_Approved'].replace({0:1, 1:0, 2:1})
  df['Approved'] = df['Delivery_Approved'] & df['Bureaucratic_Code_Approved']

  #Cambio TRF por una columna que es el valor medio de los TRF de la oportunidad
  df["TRF"] = df.groupby("Opportunity_ID")["TRF"].transform("mean")

  def combineProducts(x):
    products = ""
    added = []
    for product in x:
      product = re.findall('\d+', product)[0]
      if added.count(product) == 0:
        products += (product)
        added.append(product)
    return products

  #Junto todos los productos en una sola entrada
  df['Products'] = df.groupby('Opportunity_ID')['Product_Family'].transform(combineProducts)

  #Pruebo dropear duplicados, solo cambia el producto. Si el producto no importa
  #entonces dropear duplicados no deberia importar. Obviamente dropeo el producto en el que
  #quede tambien.
  df.drop_duplicates('Opportunity_Name',inplace=True)
  df.drop(columns=['Product_Name','Product_Family','Opportunity_Name'],inplace=True)

  #Normalizo las columnas numericas
  normalized_columns = ['ASP_converted','TRF','Total_Taxable_Amount', 'Product_Amount', 'Delta_Time']
  for column in normalized_columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

  #Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
  df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                  'Prod_Category_A', 'ID', 'Opportunity_ID', 
                   'Actual_Delivery_Date'],inplace=True)

  #Dropeamos las columnas que no utilizamos
  df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                   'Last_Modified_Date','Planned_Delivery_Start_Date',
                   'Planned_Delivery_End_Date',
                   'Month', 'Delivery_Year','Price','ASP','ASP_Currency',
                   'Total_Amount_Currency',
                   'Total_Amount','Total_Taxable_Amount_Currency','Currency',
                   'Product_Category_B','Last_Modified_By', 'Account_Owner',
                   'Opportunity_Owner','Account_Name','Product_Type','Size',
                   'Billing_Country', 'Pricing_Delivery_Terms_Quote_Appr',
                   'Pricing_Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
                   'Bureaucratic_Code_0_Approved',
                   'Approved','Territory', "Sales_Contract_No","Product", "Products"]
                   ,inplace=True)

  return df

In [ ]:
df = pd.read_csv("Train_TP2_Datos_2020-2C.csv") 
df = preprocess_dataframe(df)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
#Setup para el OneHot Encoding

enc = preprocessing.OneHotEncoder()

categorical_columns = ["Region","Bureaucratic_Code","Source","Account_Type",
                       "Opportunity_Type","Quote_Type","Delivery_Terms","Brand",
                       "Has_Expiry_Date","Delivery_Quarter","Same_Owner"]

no_enc_df = df.copy() #Creo este aux para mantener el encoding para las predicciones

In [ ]:
#Creo un df de cada columna categorica y lo joineo con el df original

for feature in categorical_columns:
  enc.fit(df[[feature]])
  enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray())
  enc_df = enc_df.add_prefix(feature + "_")
  df = df.join(enc_df)

df.drop(columns=categorical_columns,inplace=True)#Dropeo las columnas originales

In [ ]:
df.head()

,ASP_converted,TRF,Total_Taxable_Amount,Stage,Delta_Time,Product_Amount,Delivery_Approved,Bureaucratic_Code_Approved,Region_0,Region_1,Region_2,Region_3,Region_4,Bureaucratic_Code_0,Bureaucratic_Code_1,Bureaucratic_Code_2,Bureaucratic_Code_3,Bureaucratic_Code_4,Bureaucratic_Code_5,Bureaucratic_Code_6,Source_0,Source_1,Source_2,Source_3,Source_4,Source_5,Source_6,Source_7,Source_8,Source_9,Source_10,Source_11,Source_12,Source_13,Account_Type_0,Account_Type_1,Account_Type_2,Account_Type_3,Account_Type_4,Account_Type_5,...,Opportunity_Type_6,Opportunity_Type_7,Opportunity_Type_8,Opportunity_Type_9,Opportunity_Type_10,Opportunity_Type_11,Opportunity_Type_12,Opportunity_Type_13,Opportunity_Type_14,Opportunity_Type_15,Opportunity_Type_16,Opportunity_Type_17,Opportunity_Type_18,Opportunity_Type_19,Opportunity_Type_20,Opportunity_Type_21,Opportunity_Type_22,Opportunity_Type_23,Opportunity_Type_24,Quote_Type_0,Quote_Type_1,Delivery_Terms_0,Delivery_Terms_1,Delivery_Terms_2,Delivery_Terms_3,Delivery_Terms_4,Delivery_Terms_5,Delivery_Terms_6,Delivery_Terms_7,Delivery_Terms_8,Brand_0,Brand_1,Has_Expiry_Date_0,Has_Expiry_Date_1,Delivery_Quarter_0,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Same_Owner_0,Same_Owner_1
0,0.215056,0.482815,0.547199,0,0.762596,-0.427738,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.227577,-0.204252,-0.172044,1,-0.430272,-0.427738,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.095303,-0.204252,-0.168476,1,-0.471406,-0.427738,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.150657,0.757642,0.724658,0,0.721462,-0.427738,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.150657,1.513417,1.447064,0,0.083895,-0.427738,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
target = df["Stage"]
df.drop(columns=["Stage"],inplace=True)

#Normalizo las columnas

for column in df.columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

df.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.05) #Separa el df en sets de "training" y validacion

In [ ]:
log_loss_min = 100
curr_log_loss = 0
best_accuracy = 0
curr_accuracy = 0
best_n = 0

plot_points = [] #Para despues hacer el grafico de K vs LogLoss

model = KNeighborsClassifier()
best_model = model #Creo este aux para despues hacer las predicciones, ya que sino model queda con el ultimo i

for n in range(2,100):
  print("n =", n)
  model = KNeighborsClassifier(n_neighbors=n)
  model.fit(X_train,y_train)
  y_pred = model.predict_proba(X_test)
  curr_log_loss = metrics.log_loss(y_test, y_pred)
  plot_points.append(curr_log_loss)
  print(curr_log_loss)
  if (curr_log_loss < log_loss_min):
        log_loss_min = curr_log_loss
        best_n = n
        best_model = model


print(best_n, log_loss_min)

n = 2
7.289209510584202
n = 3
3.7181107305482626
n = 4
2.5207833407542983
n = 5
1.4708530713146348
n = 6
1.2165877536533216
n = 7
1.0162517355752094
n = 8
0.9460777814205956
n = 9
0.7516027042965873
n = 10
0.7508454750015473
n = 11
0.7408460233638178
n = 12
0.7399637021859019
n = 13
0.7377466343671444
n = 14
0.7332554491297246
n = 15
0.7322281537049098
n = 16
0.7291963718692626
n = 17
0.7283130785746985
n = 18
0.662035425340449
n = 19
0.6613611330735815
n = 20
0.6671029337678424
n = 21
0.6703834545152251
n = 22
0.6691340549822401
n = 23
0.6729688237179619
n = 24
0.6723533035342577
n = 25
0.6734424555903763
n = 26
0.6726713352221836
n = 27
0.6745351270869764
n = 28
0.6766445537823693
n = 29
0.6771628993299565
n = 30
0.6765780962518408
n = 31
0.6795315299532336
n = 32
0.6144924572522064
n = 33
0.6138647922808252
n = 34
0.6128907036506732
n = 35
0.6100703149820939
n = 36
0.6090586316376777
n = 37
0.6093043794781893
n = 38
0.610499495262455
n = 39
0.6114577359428573
n = 40
0.60920317588089

In [ ]:
#Grafico de K vs LogLoss

plt.figure(figsize=(12, 6))
plt.xticks(range(2,101,5))
plt.ylim(log_loss_min - 0.05,1)
plt.xlim(2,100)
plt.plot(range(2, 100), plot_points, color='red', linestyle='dashed', marker='o', markerfacecolor='blue', markersize=10)
plt.title('Log Loss vs K Neighbours')
plt.xlabel('K')
plt.ylabel('Log Loss')
plt.savefig("k vs logloss")

In [ ]:
frio_test_df = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
frio_test_df['Stage'] = 'Closed Won' #Esto esta solo para que haga el preprocess, pero al final se dropea esta columna
aux_df = frio_test_df[['Opportunity_ID']] #Esta columna la dropea el preprocesado sino
aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True)

frio_test_df = preprocess_dataframe(frio_test_df)

frio_test_df.reset_index(inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
for feature in categorical_columns:
  enc.fit(no_enc_df[[feature]])
  enc_df = pd.DataFrame(enc.transform(frio_test_df[[feature]]).toarray())
  enc_df = enc_df.add_prefix(feature + "_")
  frio_test_df = frio_test_df.join(enc_df)

frio_test_df.drop(columns=categorical_columns + ["index"],inplace=True)#Ver si dropeo brand
frio_test_df.drop(columns=["Stage"],inplace=True)

#Normalizo las columnas

for column in df.columns:
    frio_test_df[column] = (frio_test_df[column] - frio_test_df[column].mean()) / frio_test_df[column].std()

frio_test_df.fillna(1, inplace=True)#Pasa que Quote Type tiene todos los valores en 1, entonces std() da 0 por lo que la normalizacion queda en NaN

In [ ]:
test_features = frio_test_df.values

predictions = model.predict_proba(test_features)
aux_df['Target'] = predictions[:,1]
aux_df.to_csv('prediccionesFrioFrioSinTest.csv', index=False)